In [48]:
import requests
import pandas as pd
from IPython.display import HTML,clear_output, display

# Ссылка на репозиторий
repo_link = "https://github.com/krishnadey30/LeetCode-Questions-CompanyWise"
owner, repo = repo_link.split('/')[-2:]

# Базовые URL для API и raw-файлов
api_url = f"https://api.github.com/repos/{owner}/{repo}/contents"
raw_base_url = f"https://raw.githubusercontent.com/{owner}/{repo}/master"

# Получение содержимого корневой директории
def get_repo_contents(path=""):
    url = f"{api_url}/{path}" if path else api_url
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

# Рекурсивный поиск CSV-файлов
def find_csv_files(contents, base_path=""):
    csv_files = {}
    for item in contents:
        path = item["path"]
        if path.lower().endswith(".csv"):
            if path.lower().split('_')[0] in csv_files.keys():
                csv_files[path.lower().split('_')[0]].append(path)
            else:
                csv_files[path.lower().split('_')[0]] = [path]
            
        
    return csv_files

# Основной код
try:
    # 1. Получаем список всех CSV-файлов
    contents = get_repo_contents()
    csv_files = find_csv_files(contents)

    clear_output(wait=True)
    if not csv_files:
        print("CSV-файлы не найдены.")
    else:
        # 2. Выводим список файлов для выбора
        print("\n\nДоступные компании:")
        for idx, file in enumerate(list(csv_files.keys()), 1):
            print(f"{idx}. {file}")

        # 3. Выбор файла
        clear_output(wait=True)
        choice = int(input("Введите номер файла для отображения: ")) - 1
        
        if 0 <= choice < len(list(csv_files.keys())):
            selected_files = csv_files[list(csv_files.keys())[choice]]
            

            
            print("\n\nДоступные CSV-файлы:")
            for idx, file in enumerate(selected_files, 1):
                print(f"{idx}. {file}")
            select = int(input("Выберите опрделенный файл: ")) - 1
            if 0 <= select < len(selected_files):
                
                raw_url = f"{raw_base_url}/{selected_files[select]}"

            # 4. Загрузка в pandas
            df = pd.read_csv(raw_url)
            df['Title'] = '<a href="' + df['Leetcode Question Link'].transform(str.strip) + '">' + df['Title'] + '</a>'
            df.set_index('ID', inplace=True)
            html_table = df[[i for i in df.columns if i !='Leetcode Question Link']].to_html(escape=False)
            clear_output(wait=True)
            display(HTML(html_table))
        else:
            print("Неверный номер файла.")
except requests.exceptions.RequestException as e:
    print(f"Ошибка запроса: {e}")
except Exception as e:
    print(f"Произошла ошибка: {e}")

,Title,Acceptance,Difficulty,Frequency
ID,,,,
146,LRU Cache,33.2%,Medium,0.460802
228,Summary Ranges,39.5%,Medium,0.442178
206,Reverse Linked List,62.5%,Easy,0.441932
125,Valid Palindrome,36.7%,Easy,0.378728
22,Generate Parentheses,62.7%,Medium,0.377582
1,Two Sum,45.6%,Easy,0.360567
283,Move Zeroes,57.8%,Easy,0.358814
20,Valid Parentheses,39.0%,Easy,0.316136
49,Group Anagrams,56.9%,Medium,0.310519
